## RAG_Clova

#### Env

Python Version : Python 3.12.2

requirements : 
https://ssl.pstatic.net/static/clova/service/hyperclova/cookbook/rag/requirements.txt

In [1]:
# Vector DB인 Milvus와 관련된 모듈들은 모듈의 용도를 명확히 구분하기 위해 Vector DB 구축 단계에서 불러왔으며 해당 부분에서 코드를 확인하실 수 있습니다.
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests
import dotenv
dotenv.load_dotenv()
CLOVA_api_key=os.getenv("X-NCP-CLOVASTUDIO-API-KEY")
NCP_API_api_key=os.getenv("X-NCP-APIGW-API-KEY")
clova_REQUEST_ID=os.getenv("X-NCP-CLOVASTUDIO-REQUEST-ID")

#### 1. Raw Data → Connecting

In [2]:
url_to_filename_map = {}
wget_path="/opt/homebrew/bin/wget"
with open("langchainwikiurl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]
 
folder_path = "langchainwikiguide"
 
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    subprocess.run([wget_path, "-O", file_path, url], check=True)
    url_to_filename_map[url] = filename
 
with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)

--2024-08-25 23:24:39--  https://wikidocs.net/233341
Resolving wikidocs.net (wikidocs.net)... 15.165.86.219
Connecting to wikidocs.net (wikidocs.net)|15.165.86.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99984 (98K) [text/html]
Saving to: ‘langchainwikiguide/233341.html’

     0K .......... .......... .......... .......... .......... 51% 6.59M 0s
    50K .......... .......... .......... .......... .......   100% 58.6M=0.008s

2024-08-25 23:24:39 (11.6 MB/s) - ‘langchainwikiguide/233341.html’ saved [99984/99984]

--2024-08-25 23:24:39--  https://wikidocs.net/233342
Resolving wikidocs.net (wikidocs.net)... 15.165.86.219
Connecting to wikidocs.net (wikidocs.net)|15.165.86.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99828 (97K) [text/html]
Saving to: ‘langchainwikiguide/233342.html’

     0K .......... .......... .......... .......... .......... 51% 6.69M 0s
    50K .......... .......... .......... .......... .......   100

##### 보안 에러 나는 경우 방법

In [15]:
import os
import json
import requests
from tqdm import tqdm

url_to_filename_map = {}
folder_path = "langchainwikiguide"

with open("langchainwikiurl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

for url in tqdm(urls, desc="Downloading"):
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        
        url_to_filename_map[url] = filename

    except requests.RequestException as e:
        print(f"An error occurred while downloading {url}: {e}")

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file, indent=4)

print("Download complete. Mapping saved to url_to_filename_map.json")


Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.46it/s]

Download complete. Mapping saved to url_to_filename_map.json


##### Quote

txt 파일을 작성할 때는 한 줄에 하나의 URL을 작성해야 합니다. 이때, 각 URL은 줄바꿈으로 구분되어야 합니다.

그리고 html 데이터를 로딩하기 전, robots.txt를 통해 데이터로 활용할 사이트의 접근 허용 여부를 확인해야합니다. NCP의 클로바스튜디오 사용 가이드의 경우 User-agent: * Allow:/ 로 접근이 가능합니다.

예제에서 활용한 HTML은, 네이버클라우드 플랫폼(NCP)의 클로바스튜디오 사용 가이드중 CLOVA Studio("AI Services" → "CLOVA Studio")와 관련된 모든 안내 페이지를 데이터로 활용했습니다.

LangChain을 활용해 로딩한 html은, 파일을 저장한 디렉토리의 주소를 metadata의 'source'로 가져오게 됩니다. 추후 답변 제공시, 디렉토리 주소가 아닌 실제 URL을 제공하기 위해, LangChain이 로딩한 데이터를 수정해야합니다.

html을 로딩할 때 파일명을 URL로 할 경우, /와 : 기호로 인해 파일명이 깨지게 됩니다. 따라서 원본 URL과 로딩한 HTML 파일을 쌍으로 mapping하고, 이 정보를 json 형식으로 "url_to_filename_map"에 저장합니다.

이후, html 파일의 저장 경로가 담긴 'source'를, 이 json 파일과 연결해 실제 URL로 변환해줍니다. 위 코드는, json 파일을 저장하는 단계까지입니다.

##### LangChain 활용 HTML 로딩



In [16]:
# 폴더 이름에 맞게 수정
html_files_dir = Path('langchainwikiguide')
 
html_files = list(html_files_dir.glob("*.html"))
 
langchainwikidatas = []
 
for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    langchainwikidatas.append(document_data)
    print(f"Processed {html_file}")

Processed langchainwikiguide/233341.html
Processed langchainwikiguide/233346.html
Processed langchainwikiguide/233344.html
Processed langchainwikiguide/233345.html
Processed langchainwikiguide/250954.html
Processed langchainwikiguide/233342.html
Processed langchainwikiguide/233343.html


##### Quote

각 HTML에는 page_content에 모든 텍스트가, metadata의 'source'에는 저장 경로가 기록되어 있습니다. 위는, 로딩된 데이터 중 하나인 clovastudio-info.html의 형태입니다. metadata의 'source'에 실제 URL이 아닌 html 파일이 저장된 디렉토리 경로가 담긴 것을 확인할 수 있습니다. Mapping을 통해 이 'source'를 실제 URL로 바꾸는 작업을 다음 단계에 진행하게 됩니다.

##### Mapping 정보를 활용해 'source'를 실제 URL로 대체

In [17]:
with open("url_to_filename_map.json", "r") as map_file:
    url_to_filename_map = json.load(map_file)
 
filename_to_url_map = {v: k for k, v in url_to_filename_map.items()}
 
# langchainwikidatas 리스트의 각 Document 객체의 'source' 수정
for doc_list in langchainwikidatas:
    for doc in doc_list:
        extracted_filename = doc.metadata["source"].split("/")[-1]
        if extracted_filename in filename_to_url_map:
            doc.metadata["source"] = filename_to_url_map[extracted_filename]
        else:
            print(f"Warning: {extracted_filename}에 해당하는 URL을 찾을 수 없습니다.")

In [18]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
langchainwiki_flattened = [item for sublist in langchainwikidatas for item in sublist]

In [19]:
langchainwiki_flattened 

[Document(page_content="<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기 01. OpenAI API 키 발급 및 테스트 02. LangSmith 추적 설정 03. OpenAI API 사용(GPT-4o 멀티모달) 04. LangChain Expression Language(LCEL) 05. LCEL 인터페이스 06. Runnable CH02 프롬프트(Prompt) 01. 프롬프트(Prompt) 02. 퓨샷 프롬프트(FewShotPromptTemplate) 03. LangChain Hub 04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers) 01. Pydantic 출력 파서(PydanticOutputParser) 02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser) 03. 구조화된 출력 파서(StructuredOuputParser) 04. JSON 출력 파서(JsonOutputParser) 05. 데이터프레임 출력 파서(PandasDataFrameOutputParser) 06. 날짜 형식 출력 파서(DatetimeOutputParser) 07. 열거형 출력 파서(EnumOutputParser) 08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model) 01. 다양한 LLM 모델 활용 02. 캐싱(Cache) 03. 모델 직렬화(Serialization) - 저장 및 불러오기 04. 토큰 사용량 확인 05. 구글 생성 AI(Google Generative AI) 06. 허깅페이스 엔드포인트(HuggingFace Endpoints) 07. 허깅페이스 로컬(HuggingFace Local) 08. 허깅페이스 파이프라인(HuggingFace Pipeline) 09. 올라마(Ollama) 10. GPT4ALL CH05 메모리(Memory) 01. 대화 버퍼 메모리(ConversationBufferM

#### 2. Chunking



임베딩 모델이 처리할 수 있는 적당한 크기로 raw data를 나누는 것은 매우 중요합니다. 이는 임베딩 모델마다 한 번에 처리할 수 있는 토큰 수의 한계가 있기 때문입니다. CLOVA Studio의 문단 나누기 API는 모델이 직접 문장들간의 의미 유사도를 찾아 최적의 chunk 개수와 사용자가 원하는 1개 chunk의 크기(글자 수)를 직접 설정하여 문단을 나눌 수도 있습니다. 추가로, 후처리(postProcess = True)를 통해 chunk당 글자 수의 상한선과 하한선을 postProcessMaxSize와 postProcessMinSize로 조절할 수도 있습니다.

In [21]:
class SegmentationExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }
 
        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            "POST",
            "/testapp/v1/api-tools/segmentation/90972c6f0a784c9886f4bc6ae8a66c01", # If using Service App, change 'testapp' to 'serviceapp', and corresponding app id.
            json.dumps(completion_request),
            headers
        )
        response = conn.getresponse()
        print(response.length)
        result = json.loads(response.read().decode(encoding="utf-8"))
        print(result)
        conn.close()
        return result
 
    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["topicSeg"]
        else:
            raise ValueError(f"{res}")
 
 
if __name__ == "__main__":
    segmentation_executor = SegmentationExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key="NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/",
        api_key_primary_val="J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB",
        request_id=""
    )
 
    chunked_html = []
 
    for htmldata in tqdm(langchainwiki_flattened ):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": 1.5,
                "segCnt": -1,
                "postProcessMinSize": 0,
                "text": htmldata.page_content,
                "postProcess": False
            }

            request_json_string = json.dumps(request_data)
            request_data = json.loads(request_json_string, strict=False)
            print(1)
            response_data = segmentation_executor.execute(request_data)
        except Exception as e:
            print(f"Error occurred. Message: {e}")
        
        for paragraph in response_data:
            chunked_document = {
                "source": htmldata.metadata["source"],
                "text": paragraph
            }
            chunked_html.append(chunked_document)
 
print(len(chunked_html))

  0%|          | 0/7 [00:00<?, ?it/s]

1


 14%|█▍        | 1/7 [00:02<00:14,  2.41s/it]

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. OpenAI API 키 발급 및 테스트'], ['02. LangSmith 추적 설정'], ['03. OpenAI API 사용(GPT-4o 멀티모달)'], ['04. LangChain Expression Language(LCEL)'], ['05. LCEL 인터페이스'], ['06. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative AI)'], ['06. 허깅페이스 엔드포

 29%|██▊       | 2/7 [00:06<00:16,  3.40s/it]

...

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. OpenAI API 키 발급 및 테스트'], ['02. LangSmith 추적 설정'], ['03. OpenAI API 사용(GPT-4o 멀티모달)'], ['04. LangChain Expression Language(LCEL)'], ['05. LCEL 인터페이스'], ['06. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative AI)'], ['06. 허깅페이스 엔드포

 86%|████████▌ | 6/7 [00:20<00:03,  3.14s/it]

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. OpenAI API 키 발급 및 테스트'], ['02. LangSmith 추적 설정'], ['03. OpenAI API 사용(GPT-4o 멀티모달)'], ['04. LangChain Expression Language(LCEL)'], ['05. LCEL 인터페이스'], ['06. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative AI)'], ['06. 허깅페이스 엔드포

100%|██████████| 7/7 [00:25<00:00,  3.64s/it]

None
{'status': {'code': '20000', 'message': 'OK'}, 'result': {'topicSeg': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'], ['01. OpenAI API 키 발급 및 테스트'], ['02. LangSmith 추적 설정'], ['03. OpenAI API 사용(GPT-4o 멀티모달)'], ['04. LangChain Expression Language(LCEL)'], ['05. LCEL 인터페이스'], ['06. Runnable CH02 프롬프트(Prompt)'], ['01. 프롬프트(Prompt)'], ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'], ['03. LangChain Hub'], ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'], ['01. Pydantic 출력 파서(PydanticOutputParser)'], ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'], ['03. 구조화된 출력 파서(StructuredOuputParser)'], ['04. JSON 출력 파서(JsonOutputParser)'], ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'], ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'], ['07. 열거형 출력 파서(EnumOutputParser)'], ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'], ['01. 다양한 LLM 모델 활용'], ['02. 캐싱(Cache)'], ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'], ['04. 토큰 사용량 확인'], ['05. 구글 생성 AI(Google Generative AI)'], ['06. 허깅페이스 엔드포

In [23]:
len(chunked_html)

1486

In [24]:
chunked_html

[{'source': 'https://wikidocs.net/233341',
  'text': ['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['01. OpenAI API 키 발급 및 테스트']},
 {'source': 'https://wikidocs.net/233341', 'text': ['02. LangSmith 추적 설정']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['03. OpenAI API 사용(GPT-4o 멀티모달)']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['04. LangChain Expression Language(LCEL)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['05. LCEL 인터페이스']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['06. Runnable CH02 프롬프트(Prompt)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['01. 프롬프트(Prompt)']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['02. 퓨샷 프롬프트(FewShotPromptTemplate)']},
 {'source': 'https://wikidocs.net/233341', 'text': ['03. LangChain Hub']},
 {'source': 'https://wikidocs.net/233341',
  'text': ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)']},
 {'source': 'h

#### 3. Embedding

문단 나누기 API를 통해 나누어진 525개의 chunk(chunked_html)을 CLOVA Studio의 임베딩 API를 사용해 1024차원의 벡터로 변환하는 과정입니다. clir-emb-dolphin의 경우, 임베딩 과정에서 유사도 판단을 위해 벡터의 내적(Inner Product, IP)을 거리 단위로 사용합니다. 반면, clir-sts-dolphin은 코사인 거리(Cosine)를 거리 단위로 사용합니다. 이후, 벡터의 인덱싱과 검색 과정에서 사용자는 어떤 거리 단위를 사용하여 데이터와 사용자의 쿼리 간 유사도를 판단할 것인지 선택할 수 있습니다.

거리 단위를 임베딩부터 인덱싱, 검색까지 일치시켜야 데이터의 품질이 향상되므로, 임베딩 과정에서 사용한 모델(emb / sts)을 기억하는 것이 중요합니다. 임베딩 클래스에 오류가 발생시 멈추게끔 하는 로직을 일부 추가해, 다량의 데이터를 처리할 때 오류 발생시 재실행의 부담을 줄입니다.

In [28]:
class EmbeddingExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            "POST",
            "/testapp/v1/api-tools/embedding/v2/95709fb7cc5047b6adad8f4d8442bf24",
            json.dumps(completion_request),
            headers
        )
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding="utf-8"))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["embedding"]
        else:
            error_code = res["status"]["code"]
            error_message = res.get("status", {}).get("message", "Unknown error")
            raise ValueError(f"오류 발생: {error_code}: {error_message}")

if __name__ == "__main__":
    embedding_executor = EmbeddingExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key='NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/',
        api_key_primary_val='J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB',
        request_id='1037451f-6a88-40ae-afd4-fbc4a26cb354'
    )

    for i, chunked_document in enumerate(tqdm(chunked_html)):
        if i==10:
            break
        try:
            text_content = chunked_document['text']
            if isinstance(text_content, list):
                text_content=text_content[0]
            request_json = {
                "text": text_content
            }
            response_data = embedding_executor.execute(request_json)
            chunked_document["embedding"] = response_data
        except ValueError as e:
            print(f"Embedding API Error. {e}")
            print(chunked_document['text'])
        except Exception as e:
            print(f"Unexpected error: {e}")
        


  4%|▍         | 64/1486 [00:08<01:56, 12.22it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['02. 캐시 임베딩(CacheBackedEmbeddings)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['03. 허깅페이스 임베딩(HuggingFace Embeddings)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['04. UpstageEmbeddings']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['05. OllamaEmbeddings']


  4%|▍         | 66/1486 [00:08<01:42, 13.85it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['06. GPT4ALL 임베딩']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['07. Llama CPP 임베딩 CH09 벡터저장소(VectorStore)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['01. Chroma']


  5%|▍         | 71/1486 [00:08<01:31, 15.52it/s]

...

 11%|█         | 158/1486 [00:14<01:10, 18.78it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['06. 날짜 형식 출력 파서(DatetimeOutputParser)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['07. 열거형 출력 파서(EnumOutputParser)']
Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)']


 11%|█         | 160/1486 [00:14<01:57, 11.25it/s]

Embedding API Error. 오류 발생: 42901: Too many requests - rate exceeded
['01. 다양한 LLM 모델 활용']


KeyboardInterrupt: 

In [11]:
dimension_set = set()
 
for item in chunked_html:
    if "embedding" in item:
        dimension = len(item["embedding"])
        dimension_set.add(dimension)
 
print("임베딩된 벡터들의 차원:", dimension_set)

임베딩된 벡터들의 차원: {308}


In [14]:
chunked_html[1]

{'source': 'https://wikidocs.net/233341',
 'text': ['01. OpenAI API 키 발급 및 테스트'],
 'embedding': [['<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷 CH01 LangChain 시작하기'],
  ['01. OpenAI API 키 발급 및 테스트'],
  ['02. LangSmith 추적 설정'],
  ['03. OpenAI API 사용(GPT-4o 멀티모달)'],
  ['04. LangChain Expression Language(LCEL)'],
  ['05. LCEL 인터페이스'],
  ['06. Runnable CH02 프롬프트(Prompt)'],
  ['01. 프롬프트(Prompt)'],
  ['02. 퓨샷 프롬프트(FewShotPromptTemplate)'],
  ['03. LangChain Hub'],
  ['04. 개인화된 프롬프트(Hub에 업로드) CH03 출력 파서(Output Parsers)'],
  ['01. Pydantic 출력 파서(PydanticOutputParser)'],
  ['02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)'],
  ['03. 구조화된 출력 파서(StructuredOuputParser)'],
  ['04. JSON 출력 파서(JsonOutputParser)'],
  ['05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)'],
  ['06. 날짜 형식 출력 파서(DatetimeOutputParser)'],
  ['07. 열거형 출력 파서(EnumOutputParser)'],
  ['08. 출력 수정 파서(OutputFixingParser) CH04 모델(Model)'],
  ['01. 다양한 LLM 모델 활용'],
  ['02. 캐싱(Cache)'],
  ['03. 모델 직렬화(Serialization) - 저장 및 불러오기'],
  ['04. 토큰 

#### 4. Vector DB

CLOVA Studio의 임베딩 API를 활용하여 모든 chunked_의 text를 1024차원의 벡터로 변환한 후, 'embedding'이라는 이름의 객체로 chunked_html에 추가해 주었습니다. 이제 이 데이터를 Vector DB에 저장하고, 인덱싱을 하는 단계입니다. Vector DB로는 Milvus를 사용하였으며, 대시보드는 Docker를 사용하여 DB를 켜고(Run) 끄는(Stop) 작업을 수행하였습니다.